In [1]:
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms as T
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Add parent directory to path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from config import config as cfg

In [6]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/Users/Russel/myProjects/dl-for-blockchain/code/dl-for-blockchain-1f8fa2978dfe.json'

In [7]:
from google.cloud import bigquery as bq

In [15]:
query = ("SELECT address, bytecode from `bigquery-public-data.ethereum_blockchain.contracts` "
         'WHERE bytecode is not null '
         "LIMIT 10"
        )

In [16]:
client = bq.Client()
query_job = client.query(query,
                         location="US")

In [18]:
for row in query_job:
    print(row, "\n\n")

Row(('0xbe0f8969c60e6ad34b92375091678a9ad3991990', '0x60606040523615610053576000357c0100000000000000000000000000000000000000000000000000000000900480638da5cb5b146100c2578063f2fde38b146100fb578063fa89401a1461011357610053565b6100c05b437f5d6ac9b7defd8fbb29ef2ab0f08c3744192b3777466dd98d1fd7daed5064b6db333442604051808473ffffffffffffffffffffffffffffffffffffffff168152602001838152602001828152602001935050505060405180910390a26100bd61012b565b5b565b005b6100cf600480505061019a565b604051808273ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390f35b61011160048080359060200190919050506101c0565b005b610129600480803590602001909190505061024b565b005b600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1660003073ffffffffffffffffffffffffffffffffffffffff1631604051809050600060405180830381858888f19350505050505b565b600060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b600060009054906101000a900473ffffffffffffffffffff

In [20]:
for row in query_job:
    print(row[0])

0xbe0f8969c60e6ad34b92375091678a9ad3991990
0xd0ad0ae971a3e7dff022e6bef4bab5c603ae8e43
0xcd14a06f283f96ee48f5dc680a834ad4fe7ecb54
0x30b4f02a330af243991165b33182aa1eb04b1a04
0xa4741a75a5dc7a48b09a078699a44b32becdedc5
0x6586ec9cb29c3f8ceb630d7dd1a8992d353df81a
0x0a2f6961e477cae32439d07a4c0c5e93589f966b
0x763cc868fdaad892fd6cb6585dc8ca7f0f2679a9
0x0602fa4e00ba5b05af81c96b1d530504e99375e7
0xae4e48678119586588f2c97ea2961ae13bda35f9


In [21]:
import pandas as pd

df = pd.DataFrame([(row[0], row[1]) for row in query_job], columns=['address', 'bytecode'])

In [22]:
df

address  \
0  0xbe0f8969c60e6ad34b92375091678a9ad3991990   
1  0xd0ad0ae971a3e7dff022e6bef4bab5c603ae8e43   
2  0xcd14a06f283f96ee48f5dc680a834ad4fe7ecb54   
3  0x30b4f02a330af243991165b33182aa1eb04b1a04   
4  0xa4741a75a5dc7a48b09a078699a44b32becdedc5   
5  0x6586ec9cb29c3f8ceb630d7dd1a8992d353df81a   
6  0x0a2f6961e477cae32439d07a4c0c5e93589f966b   
7  0x763cc868fdaad892fd6cb6585dc8ca7f0f2679a9   
8  0x0602fa4e00ba5b05af81c96b1d530504e99375e7   
9  0xae4e48678119586588f2c97ea2961ae13bda35f9   

                                            bytecode  
0  0x60606040523615610053576000357c01000000000000...  
1  0x60606040523615610053576000357c01000000000000...  
2  0x60606040523615610053576000357c01000000000000...  
3  0x60606040523615610053576000357c01000000000000...  
4  0x60606040523615610053576000357c01000000000000...  
5  0x60606040523615610053576000357c01000000000000...  
6  0x60606040523615610053576000357c01000000000000...  
7  0x60606040523615610053576000357c01000000000000...  
8  0x60606040523615610053576000357c01000000000000...  
9  0x60606040523615610053576000357c01000000000000...

In [25]:
from data import mining

In [26]:
%env ETHERSCAN_API_KEY='16238JDDCF47MSA3TBPE3GAZTVAHRUUH6S'

env: ETHERSCAN_API_KEY='16238JDDCF47MSA3TBPE3GAZTVAHRUUH6S'


In [56]:
result = mining.get_sourcecode('0x06012c8cf97BEaD5deAe237070F9587f8E7A266d')
print(result[0]['SourceCode'])

pragma solidity ^0.4.11;


/**
 * @title Ownable
 * @dev The Ownable contract has an owner address, and provides basic authorization control
 * functions, this simplifies the implementation of "user permissions".
 */
contract Ownable {
  address public owner;


  /**
   * @dev The Ownable constructor sets the original `owner` of the contract to the sender
   * account.
   */
  function Ownable() {
    owner = msg.sender;
  }


  /**
   * @dev Throws if called by any account other than the owner.
   */
  modifier onlyOwner() {
    require(msg.sender == owner);
    _;
  }


  /**
   * @dev Allows the current owner to transfer control of the contract to a newOwner.
   * @param newOwner The address to transfer ownership to.
   */
  function transferOwnership(address newOwner) onlyOwner {
    if (newOwner != address(0)) {
      owner = newOwner;
    }
  }

}



/// @title Interface for contracts conforming to ERC-721: Non-Fungible Tokens
/// @author Dieter Shirley <dete@axiomzen.co> (https:

In [57]:
import re

def comment_remover(text):
    def replacer(match):
        s = match.group(0)
        if s.startswith('/'):
            return " " # note: a space and not an empty string
        else:
            return s
    pattern = re.compile(
        r'//.*?$|/\*.*?\*/|\'(?:\\.|[^\\\'])*\'|"(?:\\.|[^\\"])*"',
        re.DOTALL | re.MULTILINE
    )
    return re.sub(pattern, replacer, text)

wo_comments = comment_remover(result[0]['SourceCode'])

In [59]:
wo_comments_stripped = wo_comments.replace('\r', "")
wo_comments_stripped = wo_comments_stripped.replace('\n', "")
wo_comments_stripped = wo_comments_stripped.replace('{', "")
wo_comments_stripped = wo_comments_stripped.replace('}', "")
wo_comments_stripped = wo_comments_stripped.replace('[', "")
wo_comments_stripped = wo_comments_stripped.replace(']', "")
print(wo_comments_stripped)

pragma solidity ^0.4.11; contract Ownable   address public owner;     function Ownable()     owner = msg.sender;       modifier onlyOwner()     require(msg.sender == owner);    _;       function transferOwnership(address newOwner) onlyOwner     if (newOwner != address(0))       owner = newOwner;        contract ERC721          function totalSupply() public view returns (uint256 total);    function balanceOf(address _owner) public view returns (uint256 balance);    function ownerOf(uint256 _tokenId) external view returns (address owner);    function approve(address _to, uint256 _tokenId) external;    function transfer(address _to, uint256 _tokenId) external;    function transferFrom(address _from, address _to, uint256 _tokenId) external;         event Transfer(address from, address to, uint256 tokenId);    event Approval(address owner, address approved, uint256 tokenId);                                  function supportsInterface(bytes4 _interfaceID) external view returns (bool);   cont

In [69]:
result[0]['ABI']

'[{"constant":true,"inputs":[{"name":"_interfaceID","type":"bytes4"}],"name":"supportsInterface","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"cfoAddress","outputs":[{"name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"_tokenId","type":"uint256"},{"name":"_preferredTransport","type":"string"}],"name":"tokenMetadata","outputs":[{"name":"infoUrl","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"promoCreatedCount","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"_to","type":"address"},{"name":"_tokenId","type":"uint256"}],"name":"approve